# 1. Google Drive Mount

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


# 2. 라이브러리 import

In [0]:
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn.metrics import classification_report
import math
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# 3. 데이터 불러오기
- 데이터 개수 파악하기

In [0]:
def load_data(train_dir, test_dir):
    train_data = pd.read_csv(train_dir)
    test_data = pd.read_csv(test_dir)
    
    index, count = np.unique(train_data['label'], return_counts=True)
    print(index)
    print(count)
    
    index, count = np.unique(test_data['label'], return_counts=True)
    print(index)
    print(count)

    train = train_data.to_dict(orient='records')
    test = test_data.to_dict(orient='records')

    return train, test

In [0]:
train_dir = "/content/gdrive/My Drive/Samsung_SDS_S2/Naive Bayes/train.csv"
test_dir = "/content/gdrive/My Drive/Samsung_SDS_S2/Naive Bayes/test.csv"

train, test = load_data(train_dir, test_dir)
print(train)
print(test)

['ham' 'spam']
[4334  680]
['ham' 'spam']
[491  67]
[{'label': 'spam', 'mail': "LookAtMe!: Thanks for your purchase of a video clip from LookAtMe!, you've been charged 35p. Think you can do better? Why not send a video in a MMSto 32323."}, {'label': 'ham', 'mail': "Aight, I'll hit you up when I get some cash"}, {'label': 'ham', 'mail': 'Don no da:)whats you plan?'}, {'label': 'ham', 'mail': 'Going to take your babe out ?'}, {'label': 'ham', 'mail': 'No need lar. Jus testing e phone card. Dunno network not gd i thk. Me waiting 4 my sis 2 finish bathing so i can bathe. Dun disturb u liao u cleaning ur room.'}, {'label': 'ham', 'mail': 'Probably, want to pick up more?'}, {'label': 'spam', 'mail': 'Last Chance! Claim ur 傭?50 worth of discount vouchers today! Text SHOP to 85023 now! SavaMob, offers mobile! T Cs SavaMob POBOX84, M263UZ. 傭?.00 Sub. 16'}, {'label': 'ham', 'mail': 'K sure am in my relatives home. Sms me de. Pls:-)'}, {'label': 'ham', 'mail': "The LAY MAN! Just to let you know y

# 4. P(정상메일) and P(스팸 메일) 값 구하기

In [0]:
# P(C) 확률 구하기
def make_prior_prob(train_data):
    ham_num, spam_num = 0, 0
    for i in range(0, len(train_data)):
        if train_data[i]["label"] == "ham":
            ham_num += 1
        else:
            spam_num += 1

    prob_ham = ham_num / (ham_num + spam_num)
    prom_spam = spam_num / (ham_num + spam_num)

    return prob_ham, prom_spam

In [0]:
prob_ham, prob_spam = make_prior_prob(train)    # P(ham), P(spam) 확률 구하기
print(prob_ham)
print(prob_spam)

0.864379736737136
0.13562026326286397


# 5. ham메일과 spam메일 중복 단어 카운드

In [0]:
# ham 메일, spam 메일 중복 단어 카운트 하고 ham_dic, spam_dic에 넣기
# ham 메일, spam 메일 단어수 카운트하기
def make_likelihood(train):
    ham_dic, spam_dic = {}, {}
    ham_all_count, spam_all_count = 0, 0

    for i in range(0, len(train)):
        token_list = word_tokenize(train[i]["mail"])
        if train[i]["label"] == "ham":
            for w in token_list:
                ham_all_count += 1  # ham 메일에 사용된 전체 단어수 카운팅하기

                if w in ham_dic:
                    ham_dic[w] += 1
                else:
                    ham_dic[w] = 1

        else:
            for w in token_list:
                spam_all_count += 1  # spam 메일에 사용된 전체 단어수 카운팅하기

                if w in spam_dic:
                    spam_dic[w] += 1
                else:
                    spam_dic[w] = 1

    return ham_dic, spam_dic, ham_all_count, spam_all_count

In [0]:
ham_dic, spam_dic, ham_all_count, spam_all_count = make_likelihood(train)   # 중복 단어 카운팅하고 ham메일과 spam메일에 사용된 단어 수 구하기
print(ham_dic)
print(spam_dic)
print(ham_all_count)
print(spam_all_count)

{'Aight': 26, ',': 1348, 'I': 1732, "'ll": 205, 'hit': 10, 'you': 1501, 'up': 239, 'when': 220, 'get': 259, 'some': 99, 'cash': 11, 'Don': 8, 'no': 118, 'da': 121, ':': 488, ')': 384, 'whats': 11, 'plan': 28, '?': 1608, 'Going': 15, 'to': 1406, 'take': 82, 'your': 336, 'babe': 45, 'out': 174, 'No': 123, 'need': 128, 'lar': 33, '.': 3519, 'Jus': 10, 'testing': 2, 'e': 64, 'phone': 68, 'card': 12, 'Dunno': 17, 'network': 4, 'not': 317, 'gd': 14, 'i': 832, 'thk': 37, 'Me': 22, 'waiting': 25, '4': 158, 'my': 557, 'sis': 16, '2': 277, 'finish': 31, 'bathing': 4, 'so': 267, 'can': 262, 'bathe': 10, 'Dun': 9, 'disturb': 8, 'u': 643, 'liao': 31, 'cleaning': 3, 'ur': 170, 'room': 30, 'Probably': 8, 'want': 133, 'pick': 59, 'more': 66, 'K': 44, 'sure': 58, 'am': 166, 'in': 688, 'relatives': 2, 'home': 139, 'Sms': 1, 'me': 637, 'de': 21, 'Pls': 59, '-': 148, 'The': 69, 'LAY': 1, 'MAN': 3, '!': 766, 'Just': 80, 'let': 49, 'know': 197, 'are': 285, 'missed': 16, 'and': 664, 'thought': 34, 'off': 49,

# 6. A메일이 ham메일인지 spam메일인지 예측하기
- 라플라스 스무싱 하기
- log 씌워서 값 normalization 하기

In [0]:
def predict_mail(test, ham_dic, spam_dic, han_all_count, spam_all_count, prob_ham, prob_spam):
    pred_list = []

    for i in range(0, len(test)):
        temp_sentence = test[i]["mail"]
        temp_token = word_tokenize(temp_sentence)

        # 해당 메일이 ham 메일일 확률 구하기
        ham_cal = math.log(prob_ham)  # Log(P(ham))
        for j in range(0, len(temp_token)):
            if temp_token[j] in ham_dic:
                ham_cal += math.log(((ham_dic[temp_token[j]] + 1) / han_all_count))  # Log(P(ham)) + Log(P(Word1|ham) + Log(P(Word2|ham) ...
            else:
                ham_cal += math.log((1 / han_all_count))  # train 데이터에 존재하지 않는 단어 예외 처리

        # 해당 메일이 spam 메일일 확률 구하기
        spam_cal = math.log(prob_spam)  # Log(P(spam))
        for k in range(0, len(temp_token)):
            if temp_token[k] in spam_dic:
                spam_cal += math.log(((spam_dic[temp_token[k]]+1) / spam_all_count))  # Log(P(spam)) + Log(P(Word1|spam) + Log(P(Word2|spam) ...
            else:
                spam_cal += math.log((1 / spam_all_count))  # train 데이터에 존재하지 않는 단어 예외 처리

        # 확률 비교해서 해당 메일 ham or spam으로 결정하기
        if ham_cal > spam_cal:
            pred_list.append("ham")
        else:
            pred_list.append("spam")

    return pred_list

In [0]:
pred_list = predict_mail(test, ham_dic, spam_dic, ham_all_count, spam_all_count, prob_ham, prob_spam)  # test 데이터에 메일 분류하기
print(pred_list)

['spam', 'ham', 'spam', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'spam', 'ham', 'ham', 'spam', 'ham', 'spam', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'spam', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ha

# 7. test 메일 evalutae하기

In [0]:
test_label_list = []
for i in range(0, len(test)):
    test_label_list.append(test[i]["label"])
    
print(test_label_list)
print(classification_report(test_label_list, pred_list))

['ham', 'ham', 'spam', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham', 'ham'